In [2]:
# Python 3.x
import re
import math

def Input(day):
    "Open this day's input file."
    filename = 'inputs/{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        print("Oops, couldn't open input")


# Day 1 - Inverse Captcha
## Part 1

> The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

This could be done by reading the digits into an array, then reducing them. As a Python noob, I read that for clarity, list comprehensions are preferred over lambdas, e.g. see Guido van Rossum's [post](http://www.artima.com/weblogs/viewpost.jsp?thread=98196) from 2005. However, list comprehensions produce lists, whereas reduce produces a single output. Van Rossum concedes that `reduce()` may be used when the operator is associative, so I'm going to go ahead with `reduce()`.

In [3]:
from functools import reduce

In [4]:
def chars_to_int_array(str):
    "Given a string of digits, returns them as an array of integers"
    return [int(ch) for ch in str]

def circular_list_next_item(list, index, offset=1): 
    "Returns the list item that is 'offset' elements away after the one given by the index; the item after the last item is the first item"
    return list[(index + offset) % len(list)]

def inverse_captcha_1(arr):
    "Returns sum of digits in array that match the next digit, treating list as circular."
    return reduce(lambda sum, i: sum + (arr[i] if circular_list_next_item(arr, i) == arr[i] else 0), range(len(arr)), 0)
    

assert inverse_captcha_1(chars_to_int_array("1122")) == 3
assert inverse_captcha_1(chars_to_int_array("1111")) == 4
assert inverse_captcha_1(chars_to_int_array("1234")) == 0
assert inverse_captcha_1(chars_to_int_array("91212129")) == 9

inverse_captcha_1(chars_to_int_array(Input(1).read().rstrip("\n")))

1141

## Part 2

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. Fortunately, your list has an even number of elements

In [5]:
def inverse_captcha_2(arr):
    "Returns sum of digits in array that match the digit that is halfway around, treating list as circular."
    return reduce(lambda sum, i: sum + (arr[i] if circular_list_next_item(arr, i, len(arr)//2) == arr[i] else 0), range(len(arr)), 0)

assert inverse_captcha_2(chars_to_int_array("1212")) == 6
assert inverse_captcha_2(chars_to_int_array("1221")) == 0
assert inverse_captcha_2(chars_to_int_array("123425")) == 4
assert inverse_captcha_2(chars_to_int_array("123123")) == 12
assert inverse_captcha_2(chars_to_int_array("12131415")) == 4

inverse_captcha_2(chars_to_int_array(Input(1).read().rstrip("\n")))

950

# Day 2

## Part 1
> The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

Initially, I thought I could iterate over each row, keep track of the max and min and subtract. But it's a lot easier to just call the built-in `max` and `min`. Both methods have `O(n)` running time.

In [6]:
def line_to_int_array(line):
    "Given a string with whitespace-separated numbers, parse it as an array of integers"
    return [int(n) for n in line.split()]

def lines_to_int_arrays(lines):
    "Given a multi-line string, each with whitespace-separated numbers, return an array of arrays of integers"
    return [line_to_int_array(L) for L in lines]
    
def max_subtract_min(L):
    "Return the largest element minus the smallest element"
    return max(L) - min(L)

assert(sum([max_subtract_min(L) for L in lines_to_int_arrays("5 1 9 5\n7 5 3\n2 4 6 8".split("\n"))])) == 18

sum([max_subtract_min(L) for L in lines_to_int_arrays(Input(2).readlines())])

37923

## Part 2
> It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

I had a lot of ideas of how to solve this. Clearly, we need to consider all pairs of numbers within each row. Sorting each row's numbers first, would simplify things a little, because we could always divide the smaller into the larger, by having two loops, one from each end. Or, perhaps we could apply a modulo operator, look for the single entry with zero, but that's wasteful, as we don't abort early. 

In the end, I preferred the simpler double for-loop, and having to compute the max/min within the inner block, since it didn't requiring any sorting, or reversing.

In [7]:
def quotient_of_divisible_elements(L):
    "Given a list of numbers in which there are two numbers where one exactly divides the other, return the quotient between them."
    for i in range(0, len(L)-1):
        for j in range(i+1, len(L)):
            p = max(L[i], L[j])
            q = min(L[i], L[j])
            if p % q == 0:
                return p // q

assert(sum([quotient_of_divisible_elements(L) for L in lines_to_int_arrays("5 9 2 8\n9 4 7 3\n3 8 6 5".split("\n"))])) == 9

sum([quotient_of_divisible_elements(L) for L in lines_to_int_arrays(Input(2).readlines())])

263

# Day 3
## Part 1
> Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```
> requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the [Manhattan Distance](https://en.wikipedia.org/wiki/Taxicab_geometry) between the location of the data and square 1.
> How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?


At first, I thought about all the different ways I could represent this spiral structure, and be able to get its grid coordinates in an efficient way. After racking my brain for a bit, I noticed a pattern in the bottom right elements of each concentric "ring". They were successive powers of odd numbers, e.g. `1=1*1, 9=3*3, 25=5*5`, with the number representing the side length of each ring. Also, every element in a given ring is less than or equal to that corner element (due to the counter-clockwise way the ring is constructed).

With this knowledge, my idea was to figure out on which ring the puzzle input belonged to, figure out its position on the ring, then calculate the Manhattan distance to the center.

In [8]:
input = int(Input(3).read().rstrip('\n'))

dim = math.ceil(math.sqrt(input))
dim

559

So the input sits on the ring with bottom right corner element `559*559==312481`, and where each side has 559 numbers.  `312481 - 312051 == 430` which is less than the side length, so our element sits on the bottom row of the square.

Since the bottom right corner element is the square of an odd number, the number of steps to the center is:

In [9]:
vert_steps = dim // 2
vert_steps

279

Next, to figure out the horizontal distance to the center, we need to take the absolute difference between our input number and the center element in our row. The number in our ring, that is directly below the center `1` element would be:

In [10]:
row_center = dim**2 - dim // 2
row_center

312202

So then the number of horizontal steps would be:

In [11]:
horiz_steps = abs(input - row_center)
horiz_steps

151

So our final answer is:

In [12]:
vert_steps + horiz_steps

430

## Part 2
> If we have a function that can tell us the (x,y) of each item, given the square's index, we could easily compute the values by filling them in sequentially, as we could easily get the neighbours from a 2D array.

Do the values fit a recurrence relation?
Could we figure out a way to map a straight 1d array and coil it?

The size of each concentric "ring" is:
1, 8=2*4, 16=4*4, 24=6*4


1 R 
2 U 3 L 4 L 5 D 6 D 7 R 8 R 9 R
10 U 11 U 12 U 13 L 14 L 15 L 16 L 17 D 18 D 19 D 20 D 21 R 22 R 23 R 24 R 25 R

Make a really large array, e.g. 559x559. Start in middle. Then keep track of size of spiral we're currently filling, and then fill it.



In [13]:
def initialize_2d(rows, cols, init_value = 0):
    return [ [init_value] * cols for r in range(rows) ]

def neighbour_sum(arr, x, y):
    "Return the sum of the adjacent neighbours of arr[x][y], including diagonals"
    rows = len(arr)
    cols = len(arr[0])
    
    if x == (rows // 2) and y == (cols // 2):
        return 1
    
    sum = 0
    if (x-1 >= 0):
        sum += arr[x-1][y]
        if (y-1 >= 0):
            sum += arr[x-1][y-1]
        if (y+1 <= rows-1):
            sum += arr[x-1][y+1]
    if (x+1 <= cols-1):    
        sum += arr[x+1][y]
        if (y-1 >- 0):
            sum += arr[x+1][y-1]
        if (y+1 <= rows-1):
            sum += arr[x+1][y+1]

    if (y-1 >- 0):
        sum += arr[x][y-1]
    if (y+1 <= rows-1):
        sum += arr[x][y+1]

    return sum

In [14]:
def spiral_index_generator(dim):
    generated = 0
    
    x = dim // 2
    y = dim // 2
    yield x, y

    for spiral_len in range(3, dim+1, 2):
        # Each time we start new ring, we are one up
        # from bottom right corner
        dir_steps = 1
        x += 1
        for curdir in ['U', 'L', 'D', 'R']:
            while dir_steps < spiral_len-1:
                yield x, y   

                if curdir == 'U':
                    y -= 1
                elif curdir == 'L':
                    x -= 1
                elif curdir == 'D':
                    y += 1
                elif curdir == 'R':
                    x += 1
                dir_steps += 1
                
            dir_steps = 0
        yield x, y

def stress_test(dim, sentinel):
    arr = initialize_2d(dim, dim)
    for x, y, in spiral_index_generator(dim):
        arr[x][y] = neighbour_sum(arr, x, y)
        if arr[x][y] > sentinel:
            return arr[x][y]

stress_test(15, 312051)

312453

# Day 4
## Part 1
> A passphrase consists of a series of words (lowercase letters) separated by spaces. To ensure security, a valid passphrase must contain no duplicate words. How many passphrases are valid?

The approach I thought of immediately is to maintain a map from the word to a count of the number of times the word appears on each line. But it's simpler to just put all the words in a set, which automatically removes duplicates.

In [15]:
def line_to_string_array(line):
    "Given a string with whitespace-separated words, parse it as an array of strings"
    return [n for n in line.split()]

def lines_to_string_arrays(lines):
    "Given a multi-line string, each with whitespace-separated words, return an array of arrays of words"
    return [line_to_string_array(L) for L in lines]

def duplicate_words(arr):
    "Given an array of words, return true if the array contains duplicate words, false otherwise"
    return len(arr) != len(set(arr))

assert(duplicate_words(line_to_string_array("aa bb cc dd ee"))) == False
assert(duplicate_words(line_to_string_array("aa bb cc dd aa"))) == True
assert(duplicate_words(line_to_string_array("aa bb cc dd aaa"))) == False

valid_lines = [duplicate_words(arr) for arr in lines_to_string_arrays(Input(4).readlines())]

sum([(1 if not valid else 0) for valid in valid_lines])

466

# Day 4
## Part 2
> …a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

One key insight here is that to detect an anagram, you can employ a similar technique to look for duplicate words, but sorting the letters in each word first.

In [16]:
def has_anagram(arr):
    "Given an array of strings, return true if any of the words is an anagram of any other word; false, otherwise."
    dict = { ''.join(sorted([c for c in word])): 1 for word in arr }
    return len(dict.keys()) != len(arr)

assert(has_anagram(line_to_string_array("abcde fghij"))) == False
assert(has_anagram(line_to_string_array("abcde xyz ecdab"))) == True
assert(has_anagram(line_to_string_array("a ab abc abd abf abj"))) == False
assert(has_anagram(line_to_string_array("iiii oiii ooii oooi oooo"))) == False
assert(has_anagram(line_to_string_array("oiii ioii iioi iiio"))) == True

check = [has_anagram(line) for line in lines_to_string_arrays(Input(4).readlines())]

sum(0 if has_anagram else 1 for has_anagram in check)

251

# Day 5
## Part 1
> The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

> In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

This seems fairly straightforward: read the data into a mutable array, and just update as we traverse it.

In [17]:
def lines_to_int_array(input):
    return [int(e.strip()) for e in input.split()]

def jump_maze_1(arr):
    "Given an array of integers representing jumps, follow the Part 1 rules and return the number of steps before we reach an index outside of the array dimension."
    steps = 0
    pos = 0
    while True:
        try:
            jump = arr[pos]
            arr[pos] += 1
            pos += jump
            steps += 1
        except IndexError:
            return steps

assert(jump_maze_1(lines_to_int_array("0\n 3\n 0 \n1\n -3"))) == 5

In [18]:
jump_maze_1(lines_to_int_array(Input(5).read()))

342669

## Part 2
> Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

In [19]:
def jump_maze_2(arr):
    "Given an array of integers representing jumps, follow the Part 2 rules and return the number of steps before we reach an index outside of the array dimension."
    steps = 0
    pos = 0
    while True:
        try:
            jump = arr[pos]
            offset = -1 if jump >= 3 else 1
            arr[pos] += offset
            pos += jump
            steps += 1
        except IndexError:
            return steps
assert(jump_maze_2(lines_to_int_array("0\n 3\n 0 \n1\n -3"))) == 10

In [20]:
jump_maze_2(lines_to_int_array(Input(5).read()))

25136209

# Day 6
## Part 1
> The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

> The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

The obvious solution is to just follow the description and keep track of the index of the current position in the memory bank, a counter for how many redistribution cycles have been performed, and a map of the states seen. To do a redistribution, find `max(array)` and use `array.index(max(array))` to know where to start redistribution.

Improving on the obvious solution, instead of adding one to each bank during redistribution, try to update each bank with all the blocks that it needs, by dividing the selected block value by the size of the array. There's a bit of adjustment to account for the remainder if the division is not exact.

## Part 2
> starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

This can be done by extending the redistribution implementation, and storing the cycle number where a state was first seen. When we see it again, we subtract the current cycle number from the cycle when it was first seen, to determine the size of the loop.

In [21]:
def redistribute(arr):
    alen = len(arr)
    pos = 0
    cycles = 0
    states = {}
    while True:
        most = max(arr)
        pos = arr.index(most) # finds lowest index
        arr[pos] = 0 # Remove for redistribution
        pos += 1
        while most > 0:
            arr[pos % alen] += 1
            most -= 1
            pos += 1
        cycles += 1
        #print(tuple(arr))
        if (tuple(arr) in states):
            first_repeat_state = tuple(arr)
            break
        states[tuple(arr)] = cycles # Records on which cycle this state was first seen
    return cycles, (first_repeat_state, states[first_repeat_state])

cycles, (state, cycle_first_seen) = redistribute([0,2,7,0])
assert(cycles) == 5
assert(cycles - cycle_first_seen) == 4

In [22]:
cycles, (first_repeat_state, cycle_first_seen) = redistribute(line_to_int_array(Input(6).read()))
cycles

3156

In [23]:
cycles - cycle_first_seen

1610

# Day 7
## Part 1

> One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.
> You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. 
> What is the name of the bottom program?

In [24]:
import re
def find_bottom_program(lines):
    parentmap = {}
    for line in lines:
        m = re.match(r"(\w+) \((\d+)\)( -> (.*))?", line)
        # Ignore leaf nodes, they don't help in finding lowest
        if m != None and m.group(4) != None:
            parentname = m.group(1)
            children = [e.strip() for e in m.group(4).split(',')]
            for childname in children:
                parentmap[childname] = parentname
    
    # Pick arbitrary node, traverse up to parent
    visit = list(parentmap.keys())[0]
    while visit in parentmap:
        visit = parentmap[visit]
    return visit

test_data = """
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
"""
assert(find_bottom_program(test_data.split('\n'))) == 'tknk'


In [25]:
find_bottom_program(Input(7).readlines())

'cyrupz'

## Part 2
> For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.
> Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

Store all the weights of each disc in one map. Track each disc's children in another map. Define a recursive function to compute weight of program on a disc: if leaf node (no children), weight is own weight. If children, then own weight plus sum of childrens' weights. 

In [26]:
def initialize_tower_structures(lines):
    weight_map = {}
    child_map = {}
    for line in lines:
        m = re.match(r"(\w+) \((\d+)\)( -> (.*))?", line)
        name = m.group(1)
        weight = int(m.group(2))
        weight_map[name] = weight
        
        if m.group(4) != None:
            child_map[name] = [e.strip() for e in m.group(4).split(',')]
    return weight_map, child_map

In [27]:
def program_weight(name, weight_map, child_map):
    self = weight_map[name]
    children = (sum([program_weight(child, weight_map, child_map) for child in child_map[name]])) if name in child_map else 0
    return self + children

In [28]:
weight_map, child_map = initialize_tower_structures( Input(7).readlines() )

Starting from root (found in previous part), we check weights of children and recurse on whichever has a different weight than the others. Once we have found the disc that is balanced, we look at its peers to determine what weight needs to be adjusted.

In [29]:
import collections

def find_black_sheep(L, common):
    "Given an array of items that are all assumed to be the given value 'common', return the index of the odd one out. If all elements are the same, raise StopIteration"
    return next(i for i, v in enumerate(L) if v != common)

def find_weight_adjustment(root_name, weight_map, child_map):
    print(root_name)
    child_weights = [program_weight(child, weight_map, child_map) for child in child_map[root_name]]
    print("\t{}".format(child_weights))
    try:
        common = collections.Counter(child_weights).most_common()[0][0]
        index = find_black_sheep(child_weights, common)
        next_root = child_map[root_name][index]
        adjust = find_weight_adjustment(next_root, weight_map, child_map)
        if adjust == None:
            next_root_weight = weight_map[next_root]
            black_sheep = child_weights[index]
            print("weight[{}]={} + ({} - {})".format(next_root, next_root_weight, common, black_sheep))
            return next_root_weight + (common - black_sheep)
        else:
            return adjust
    except StopIteration:
        return None

In [30]:
find_weight_adjustment('cyrupz', weight_map, child_map)

cyrupz
	[119424, 119424, 119424, 119424, 119424, 119424, 119432]
qjvtm
	[12146, 12154, 12146]
boropxd
	[1123, 1123, 1123, 1123, 1131, 1123, 1123]
cwwwj
	[310, 310, 310]
weight[cwwwj]=201 + (1123 - 1131)


193

# Day 8
## Part 1
> Each instruction consists of several parts: the register to modify, whether to increase or decrease that register's value, the amount by which to increase or decrease it, and a condition. If the condition fails, skip the instruction without modifying the register. The registers all start at 0. The instructions look like this:
```
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
```

Keep a map of all registers. Parse each instruction line, check the condition, and execute.

In [31]:
from collections import defaultdict
def parse_instruction(line):
    m = re.match(r"(\w+) (inc|dec) (-?\d+) if (\w+) (.*) (-?\d+)", line)
    return m.groups()

def test_reg_condition(regs, test_reg, oper, str_value):
    "Return the result of evaluating a register predicate."
    val = int(str_value)
    if oper == '>':
        return regs[test_reg] > val
    elif oper == '<':
        return regs[test_reg] < val
    elif oper == '>=':
        return regs[test_reg] >= val
    elif oper == '<=':
        return regs[test_reg] <= val
    elif oper == '==':
        return regs[test_reg] == val
    elif oper == '!=':
        return regs[test_reg] != val
    else:
        raise Exception
    
def compute_result(lines, regs):
    "Execute the program given in 'lines', updating register values in 'regs', and returning the largest value ever stored in any register."
    highwater = 0
    for line in lines:
        reg, inc_oper, val, test_reg, test_oper, test_value = parse_instruction(line)
        if test_reg_condition(regs, test_reg, test_oper, test_value):
            value = 0
            if inc_oper == 'inc':
                value = int(val)
            elif inc_oper == 'dec':
                value = -int(val)
            regs[reg] += value
            if regs[reg] > highwater:
                highwater = regs[reg]
    return highwater

regs = defaultdict(lambda: 0)

test = """b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10"""
highwater = compute_result(test.split('\n'), regs)
assert(max(regs.values())) == 1
assert(highwater) == 10

regs = defaultdict(lambda: 0)
compute_result(Input(8).readlines(), regs)
max(regs.values())

4066

## Part 2
> the CPU also needs to know the highest value held in any register during this process

In [32]:
regs = defaultdict(lambda: 0)
compute_result(Input(8).readlines(), regs)

4829

# Day 9
## Part 1
> The characters represent groups - sequences that begin with { and end with }. Within a group, there are zero or more other things, separated by commas: either another group or garbage. Since groups can contain other groups, a } only closes the most-recently-opened unclosed group - that is, they are nestable. Your puzzle input represents a single, large group which itself contains many smaller ones.

> Sometimes, instead of a group, you will find garbage. Garbage begins with < and ends with >. Between those angle brackets, almost any character can appear, including { and }. Within garbage, < has no special meaning… Inside garbage, any character that comes after ! should be ignored, including <, >, and even another !.

A simple state machine can be used for this. Whenever we encounter a group, we can call a "parse group" function recursively. We pass in the score from the parent group into the child group. Each parse group method returns the score for itself, the parent group adds it to its own score. We only need to process the stream a character at a time, so we can use a file/stream abstraction that keeps track of the current position.

## Part 2
Since we already wrote a method to parse garbage, extending it to keep track of the count is pretty simple. We return a tuple of the score and garbage length from the parse group method.

In [33]:
from io import StringIO
def parse_garbage(file):
    "Given a stream pointing at the character right after the start of the garbage start character, continue reading until the garbage block is completely read. The stream will be left at one character after the end garbage character. Return the number of garbage characters, not including the start/end chars nor cancelled chars nor the cancelling chars."
    count = 0
    char = read_char(file)
    while char == '!' and char != '':
        read_char(file)
        char = read_char(file)
    while char != '>' and char != '':
        count += 1
        char = read_char(file)
        while char == '!' and char != '':
            read_char(file)
            char = read_char(file)
    return count
    
def parse_group(own_score, file):
    "Given a file object, read from the current position until we see an ending group character. Return the group score."
    garbage_count = 0
    score = own_score
    char = read_char(file)
    while char != '}' and char != '':
        if char == '<':
            garbage_count += parse_garbage(file)
        elif char == '{':
            child_group_score, child_garbage_count = parse_group(own_score+1, file)
            score += child_group_score
            garbage_count += child_garbage_count
         
        char = read_char(file)

    return score, garbage_count

def read_char(file):
    char = file.read(1)
    return char

assert(parse_group(0, StringIO('{}'))[0]) == 1
assert(parse_group(0, StringIO('{{{}}}'))[0]) == 6
assert(parse_group(0, StringIO('{{},{}}'))[0]) == 5
assert(parse_group(0, StringIO('{{{},{},{{}}}}'))[0]) == 16
assert(parse_group(0, StringIO('{<a>,<a>,<a>,<a>}'))[0]) == 1
assert(parse_group(0, StringIO('{{<ab>},{<ab>},{<ab>},{<ab>}}'))[0]) == 9
assert(parse_group(0, StringIO('{{<!!>},{<!!>},{<!!>},{<!!>}}'))[0]) == 9
assert(parse_group(0, StringIO('{{<a!>},{<a!>},{<a!>},{<ab>}}'))[0]) == 3


In [34]:
parse_group(0, Input(9))

(12803, 6425)

# Day 10
## Part 1
> To achieve this, begin with a list of numbers from 0 to 255, a current position which begins at 0 (the first element in the list), a skip size (which starts at 0), and a sequence of lengths (your puzzle input). Then, for each length:

- Reverse the order of that length of elements in the list, starting with the element at the current position.
- Move the current position forward by that length plus the skip size.
- Increase the skip size by one.
> The list is circular; if the current position and the length try to reverse elements beyond the end of the list, the operation reverses using as many extra elements as it needs from the front of the list. If the current position moves past the end of the list, it wraps around to the front. Lengths larger than the size of the list are invalid.


In [35]:
def csv_line_to_int_array(line):
    "Given a string with comma-separated numbers, parse it as an array of integers"
    return [int(n) for n in line.split(',')]

def circular_list_reverse(arr, pos, length):
    "Given a circular array, and a starting position, reverse the elements from pos to pos+length-1"
    arrlen = len(arr)
    for x in range(0, length // 2):
        i = (pos + x) % arrlen
        j = (pos + length - 1 -x) % arrlen
        arr[i], arr[j] = arr[j], arr[i]
        
def knot_hash(input_lengths, arr, rounds=1):
    skip = 0
    pos = 0
    for r in range(0,rounds):
        for length in input_lengths:
            circular_list_reverse(arr, pos, length)
            pos += length + skip
            skip += 1
    return arr

In [36]:
assert(knot_hash([3,4,1,5], list(range(0, 5)))) == [3, 4, 2, 1, 0]

In [37]:
output = knot_hash(csv_line_to_int_array(Input(10).read()), list(range(0, 256)))
output[0] * output[1]

38415

## Part 2
> First, from now on, your input should be taken not as a list of numbers, but as a string of bytes instead. Unless otherwise specified, convert characters to bytes using their ASCII codes. This will allow you to handle arbitrary ASCII strings, and it also ensures that your input lengths are never larger than 255.

> Once you have determined the sequence of lengths to use, add the following lengths to the end of the sequence: 17, 31, 73, 47, 23.

> Second, instead of merely running one round like you did above, run a total of 64 rounds, using the same length sequence in each round. The current position and skip size should be preserved between rounds. 

> Once the rounds are complete, you will be left with the numbers from 0 to 255 in some order, called the sparse hash. Your next task is to reduce these to a list of only 16 numbers called the dense hash. To do this, use numeric bitwise XOR to combine each consecutive block of 16 numbers in the sparse hash (there are 16 such blocks in a list of 256 numbers). 

One thing to note about Python is the difference between `list.append()` which mutates the list by adding an object to the end, `list.extend()` which mutates a list by extending it with an iterable, and `+` which is an overload which is like `extend` but creates a new list.

In [38]:
def input_to_ascii_bytes(str):
    "Returns a list with the ASCII numeric codes of each character of the input."
    return [ord(c) for c in str]

def preprocess_p2(arr):
    "Return a new list that is the extension of the given list with some fixed values."
    return arr + [17, 31, 73, 47, 23]

assert(preprocess_p2(input_to_ascii_bytes("1,2,3"))) == [49,44,50,44,51,17, 31, 73, 47, 23]

In [39]:
def xor_list(arr):
    "Returns the bitwise XOR of the elements of the list."
    return reduce(lambda xor, i: xor ^ i, arr)
assert(xor_list([65, 27, 9, 1, 4, 3, 40, 50, 91, 7, 6, 0, 2, 5, 68, 22])) == 64

In [40]:
def dense_hash(sparse):
    "Return the bitwise XOR of groups of 16 elements from sparse, returning a list of len(sparse)/16 elements"
    return [xor_list(range) for range in [sparse[a:a+16] for a in range(0, len(sparse), 16)]]

Python's `hex()` method returns a value prefixed with `0x`. Using a format string allows us to specify padding with leading zero, number of digits and conversion type, etc.

In [41]:
def full_knot_hash(input):
    lengths = preprocess_p2(input_to_ascii_bytes(input))
    sparse_hash = knot_hash(lengths, list(range(0, 256)), 64)
    dense = dense_hash(sparse_hash)                
    return ''.join(["%0.2x" % num for num in dense])

In [42]:
assert(full_knot_hash("")) == "a2582a3a0e66e6e86e3812dcb672a272"
assert(full_knot_hash("AoC 2017")) == "33efeb34ea91902bb2f59c9920caa6cd"
assert(full_knot_hash("1,2,3")) == "3efbe78a8d82f29979031a4aa0b16a9d"
assert(full_knot_hash("1,2,4")) == "63960835bcdc130f0b66d7ff4f6a5a8e"

In [43]:
full_knot_hash(Input(10).read())

'9de8846431eef262be78f590e39a4848'

# Day 11
## Part 1
> The hexagons ("hexes") in this grid are aligned such that adjacent hexes can be found to the north, northeast, southeast, south, southwest, and northwest. You have the path the child process took. Starting where he started, you need to determine the fewest number of steps required to reach him

This one I needed help with representation and algorithms. Fortunately, Red Blog Games has a [comprehensive guide](https://www.redblobgames.com/grids/hexagons/) on hexagonal grids. I decided to use the cube coordinate system as that gave a simple algorithm.

In [44]:
def hex_grid_distance(a, b):
    "Return the distance between two points as cube coordinates on a hex grid."
    return (abs(a[0] - b[0]) + abs(a[1] - b[1]) + abs(a[2] - b[2])) / 2

def walk(steps):
    "Given a list of steps from the set of strings: [nw, n, ne, sw, s, se], map each direction using cube coordinates, and return the resulting hex grid as a cube coordinate."
    pos = (0,0,0) # (x,y,z)
    max_dist = 0
    for step in steps:
        if step == 'n':
            pos = (pos[0]+1, pos[1], pos[2]-1)
        elif step == 'nw':
            pos = (pos[0], pos[1]+1, pos[2]-1)
        elif step == 'ne':
            pos = (pos[0]+1, pos[1]-1, pos[2])
        elif step == 's':
            pos = (pos[0]-1, pos[1], pos[2]+1)
        elif step == 'sw':
            pos = (pos[0]-1, pos[1]+1, pos[2])
        elif step == 'se':
            pos = (pos[0], pos[1]-1, pos[2]+1)
        dist = hex_grid_distance((0,0,0), pos)
        if dist > max_dist:
            max_dist = dist
    return pos, max_dist

In [45]:
dest, max_dist = walk(Input(11).read().split(','))

In [46]:
hex_grid_distance((0,0,0), dest)

698.0

In [47]:
max_dist

1435.0

# Day 12
## Part 1
> You walk through the village and record the ID of each program and the IDs with which it can communicate directly (your puzzle input). Each program has one or more programs with which it can communicate, and these pipes are bidirectional; if 8 says it can communicate with 11, then 11 will say it can communicate with 8.

> You need to figure out how many programs are in the group that contains program ID 0.

This input shows the neighbours of every program. I initially searched for information about spanning trees and connected graphs, but quickly realized that the input is just an [adjacency list](https://en.wikipedia.org/wiki/Adjacency_list). A simple way to solve this part would be to just recursively add nodes as we traverse the children of ID 0. I peeked at the pseudocode in one of the answers in this [Stack Overflow post](https://stackoverflow.com/questions/21864857/find-all-connected-nodes-in-adjacency-list)! However, the examples use a directed graph, whereas ours is bidirectional, so we need to ensure we don't recurse infinitely.

In [48]:
def get_group(program_id, adjacency, g):
    "Compute the connected nodes of the given ID in the adjacency list, using a running set to keep track. Return set of nodes."
    if program_id not in g:
        g.add(program_id)
    for n in adjacency[program_id]:
        if n not in g:
            g.add(n)
            get_group(n, adjacency, g)
    return g
    
def parse_communication_list(lines):
    "Build a map of program ID communication partners, indexed by program ID. That is, the adjacency list of each program ID."
    adjacency = defaultdict(lambda: [])
    for entry in lines:
        groups = re.match(r"(\d+) <-> (.*)", entry)
        program_id = int(groups[1])
        neighbors = [int(x) for x in groups[2].split(',')]
        adjacency[program_id] = neighbors
    return adjacency

In [49]:
test = """0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5"""
adj = parse_communication_list(test.split('\n'))
assert(get_group(0, adj, set())) == set([0,2,3,4,6,5])

In [50]:
adj = parse_communication_list(Input(12).readlines())
len(get_group(0, adj, set()))

239

## Part 2
> A group is a collection of programs that can all communicate via pipes either directly or indirectly. The programs you identified just a moment ago are all part of the same group. Now, they would like you to determine the total number of groups.

A straightforward way is to maintain a set of program IDs who are not part of a group, initially, the entire group. While this set is not empty, pick one at random, compute the connected group, and remove those elements from the larger group. 

In [51]:
adj = parse_communication_list(Input(12).readlines())
all_programs = set(adj.keys())
total_groups = 0
while len(all_programs) > 0:
    group = get_group(all_programs.pop(), adj, set())
    total_groups += 1
    all_programs = all_programs.difference(group)
total_groups

215

# Day 13
## Part 1
> By studying the firewall briefly, you are able to record (in your puzzle input) the **depth** of each layer and the **range** of the scanning area for the scanner within it, written as depth: range. Each layer has a thickness of exactly 1. A layer at depth 0 begins immediately inside the firewall; a layer at depth 1 would start immediately after that.

> Visually, it might look like this:
```
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]
```
> Within each layer, a security scanner moves back and forth within its range. Each security scanner starts at the top and moves down until it reaches the bottom, then moves up until it reaches the top, and repeats. A security scanner takes **one picosecond** to move one step. 
> Your plan is to hitch a ride on a packet about to move through the firewall. The packet will travel along the top of each layer, and it moves at **one layer per picosecond**. Each picosecond, the packet moves one layer forward (its first move takes it into layer 0), and then the scanners move one step. If there is a scanner at the top of the layer **as your packet enters it**, you are **caught**. (If a scanner moves into the top of its layer while you are there, you are not caught: it doesn't have time to notice you before you leave.) 
> The **severity** of getting caught on a layer is equal to its **depth** multiplied by its **range**. (Ignore layers in which you do not get caught.)

Note that each scanner's position is independent of the others, and the only information we need to predict where a scanner will be at a given instant in time is its depth and range. Since we only care about whether we are caught or not, it suffices to figure out how many of the scanners will be at the top when we enter it.

In [52]:
def scanner_position(_d, _r, _t):
    "Given the specified scanner, with an initial position of 0 (the top) at time 0, return the scanner's position (a value between 0 and _range-1) at the *beginning* of the picosecond given by _t."

    # Period is how many time units before scanner positions repeat. The scanner goes backward
    # when it reaches the end, without repeating the last element, nor the first. So the
    # period is _r + _r - 2.
    p = (_r - 1) * 2

    # Consider the indices mod the period. So the indices would be [0, p-1] and
    # _t % p gives the correct scanner position for the first _r-1 times. For the
    # remaining ones, the position gets smaller, the further you are from _r-1.
    tmodp = _t % p
    return tmodp if tmodp < _r else p - tmodp

To calculate the penalty, we just need to get all time instances when the scanner is in the first position:

In [53]:
def severity(_d, _r):
    return _d * _r

def predict_all_future_positions(firewall, t=0):
    "Generate an array of the position of each scanner, at the instant when the packet enters that layer, starting at beginning of given time."
    return [scanner_position(d, firewall[d], t + d) if d in firewall else None for d in range(0, max(firewall.keys())+1)]

def snapshot_positions(firewall, t=0):
    "Generate an array of the position of each scanner, at the beginning of an instant in time."
    return [scanner_position(d, firewall[d], t) if d in firewall else None for d in range(0, max(firewall.keys())+1)]

def trip_severity(firewall, delay=0):
    "Calculate the total trip severity, the sum of the severities at each layer."
    pos = predict_all_future_positions(firewall, delay)
    return sum([severity(ind, firewall[ind]) if s != None and s == 0 else 0 for ind, s in enumerate(pos)])

In [54]:
test = {0: 3,1: 2,4: 4,6: 4}
assert(trip_severity(test)) == 24

In [55]:
def read_packet_scanner(lines):
    "Parse each line as a scanner definition, and return a dictionary."
    d = {}
    for line in lines:
        g = re.match(r"(\d+): (\d+)", line)
        d[int(g[1])] = int(g[2])
    return d

firewall = read_packet_scanner(Input(13).readlines())
trip_severity(firewall)

1904

## Part 2
> Now, you need to pass through the firewall without being caught - easier said than done.

> You can't control the speed of the packet, but you can delay it any number of picoseconds. For each picosecond you delay the packet before beginning your trip, all security scanners move one step. You're not in the firewall during this time; you don't enter layer 0 until you stop delaying the packet.

> Because all smaller delays would get you caught, the fewest number of picoseconds you would need to delay to get through safely is 10.

> What is the fewest number of picoseconds that you need to delay the packet to pass through the firewall without being caught?

I initially tried iterating over delay values, and checking their trip severities but my solution to the example and the full input were failing. After a bit debugging, I realized that a trip severity of **0** does not mean you don't get caught, since the penalty for getting caught in the first layer is 0.

In [56]:
def is_caught(firewall, t=0):
    "Return boolean indicating whether you would get caught traversing the firewall beginning at the specified time."
    return any(p == 0 for p in predict_all_future_positions(firewall, t))
    
def find_delay_without_caught(firewall):
    delay = 0
    caught = is_caught(firewall, delay)
    while caught:
        delay += 1
        caught = is_caught(firewall, delay)
    return delay

In [57]:
assert(find_delay_without_caught(test)) == 10

In [58]:
# The following takes a long time (~2 minutes) to get the right answer (3833504)
#find_delay_without_caught(firewall)

Potential speedups for above: there are a few layers with range 2, so they are in position 0 every other picosecond. So we could increment the delay by 2 instead of 1?

# Day 14
## Part 1
> The disk in question consists of a 128x128 grid; each square of the grid is either free or used. On this disk, the state of the grid is tracked by the bits in a sequence of knot hashes.

> A total of 128 knot hashes are calculated, each corresponding to a single row in the grid; each hash contains 128 bits which correspond to individual grid squares. Each bit of a hash indicates whether that square is free (0) or used (1).

> The hash inputs are a key string (your puzzle input), a dash, and a number from 0 to 127 corresponding to the row. For example, if your key string were flqrgnkx, then the first row would be given by the bits of the knot hash of flqrgnkx-0, the second row from the bits of the knot hash of flqrgnkx-1, and so on until the last row, flqrgnkx-127.

> Given your actual key string, how many squares are used?

This is straightforward. Use a lookup table to match each hexadecimal digit to the number of bits, then sum.

In [59]:
def bits_set(hexchar):
    return {'0':0, '1':1, '2':1, '3': 2,
            '4':1, '5':2, '6':2, '7': 3,
            '8':1, '9':2, 'a':2, 'b': 3,
            'c':2, 'd':3, 'e':3, 'f': 4}[hexchar]

def defragment_count(input):
    grid = [full_knot_hash(K) for K in ["%s-%d" % (input, i) for i in range(128)]]
    return sum([bits_set(h) for line in grid for h in line])

The syntax for a nested list comprehension (without creating a list of lists) was not obvious to me!

In [60]:
test = 'flqrgnkx'
assert(defragment_count(test)) == 8108

input = 'hfdlxzhv'
defragment_count(input)

8230

## Part 2

> A region is a group of used squares that are all adjacent, not including diagonals. Every used square is in exactly one region: lone used squares form their own isolated regions, while several adjacent squares all count as a single region.
> How many regions are present given your key string?

My initial thinking is that we can maintain a map from each bit/square's co-ordinate to their region. As we visit a square, we check to see if it has any neighbours. If so, we coalesce all of the neighbors into the same region. If not, we assign it to a new region. At the end, we can tell how many regions were needed to be assigned. A data structure for this would just be a simple map, from coordinate to region. 

First, I needed to modify some code from Part 1, to convert the knot hash output into a list of bit values. The list comprehension syntax took a while to get right: I certainly didn't write those nested expressions in one go.

In [61]:
def hex_to_bit_tuple(hexchar):
    return {'0':(0,0,0,0), '1':(0,0,0,1), '2':(0,0,1,0), '3': (0,0,1,1),
        '4':(0,1,0,0), '5':(0,1,0,1), '6':(0,1,1,0), '7': (0,1,1,1),
        '8':(1,0,0,0), '9':(1,0,0,1), 'a':(1,0,1,0), 'b': (1,0,1,1),
        'c':(1,1,0,0), 'd':(1,1,0,1), 'e':(1,1,1,0), 'f': (1,1,1,1)}[hexchar]

def make_full_grid(input):
    "Calculate the knot hashes from the input, and return a list of lists, with each element set to 0 (if square is free) or 1 (if square is used)"
    grid = [full_knot_hash(K) for K in ["%s-%d" % (input, i) for i in range(128)]]
    return [[item for sublist in [list(hex_to_bit_tuple(c)) for c in line] for item in sublist] for line in grid]

But after trying to implement my initial idea, I realized I had a problem: when I come across a node with several neighbors that belong to different regions, I need to coalesce them all into the same region. This means either a linear scan on the values of the map, or maintaining an inverse map.

Then, in the back of my mind, I recalled the [union-find algorithm](https://en.wikipedia.org/wiki/Disjoint-set_data_structure). My revised idea was to use this structure to combine adjacent regions (**union**) into the same set. I found an [implementation of union-find](https://www.nayuki.io/page/disjoint-set-data-structure) in Python from Project Nayuki. It represents disjoint sets using a number, so I first build a map (`region_map`) from the grid square to a set number.

In [62]:
from utils.disjointset import DisjointSet

def initialize_defrag_part2_structs(full_grid):
    "Assign each unique square to a unique set number, returning the map and number of sets used."
    region_map = {}
    set_num = 0
    for y in range(128):
        for x in range(128):
            if full_grid[y][x] == 1:
                region_map[(x,y)] = set_num
                set_num += 1
    return region_map, set_num

I also need a function to return the neighbours of a square. Since we're going to traverse the grid from left to right, top to bottom, we only need to look at squares we've encountered previously.

In [63]:
def get_neighbors(coord, max_coord):
    "Given a left-to-right, top-to-bottom traversal of a grid, return a list of the visited neighbors of the given coordinate. A neighbor is an adjacent square, not including diagonals."
    neighbors = []
    (x, y) = coord
    if x > 0:
        neighbors.append((x-1, y))
    if y > 0:
        neighbors.append((x, y-1))
    return neighbors

To discover the connected regions, we go through the neighbors of all used squares, and merge them into the same set. I ran into an oddity while trying to iterate a Python `defaultdict`, getting the error `dictionary changed size during iteration`. I realized I didn't need the default behavior, since the only neighbors we care about, are the ones that are used. So I switched to a regular dictionary for `region_map` and the problem went away.

In [64]:
def build_disjoint_sets(region_map, disjoint_set):
    for coord, my_region in region_map.items():
        neighbors = get_neighbors(coord, 128)
        neighbor_regions = set([region_map[k] if k in region_map else None for k in neighbors]) - {None}
        for n in neighbor_regions:
            disjoint_set.merge_sets(my_region, n)

In [65]:
def day14_part2(input):
    grid = make_full_grid(input)
    region_map, num_sets = initialize_defrag_part2_structs(grid)
    disjoint_set = DisjointSet(num_sets)
    build_disjoint_sets(region_map, disjoint_set)
    return disjoint_set.get_num_sets()

assert(day14_part2('flqrgnkx')) == 1242

my_input = 'hfdlxzhv'
day14_part2(my_input)

1103

# Day 15
## Part 1
> The generators, called generator A and generator B, are trying to agree on a sequence of numbers. 
> As they do this, a judge waits for each of them to generate its next value, compares the lowest 16 bits of both values, and keeps track of the number of times those parts of the values match.
> The generators both work on the same principle. To create its next value, a generator will take the previous value it produced, multiply it by a factor (generator A uses 16807; generator B uses 48271), and then keep the remainder of dividing that resulting product by 2147483647. That final remainder is the value it produces next.
> Suppose that for starting values, generator A uses 65, while generator B uses 8921.

The naive implementation of this is simple to implement, but it takes about a minute on my machine to run and get the right answer. The value 2147483647 is all ones in binary, and I thought this could be exploited somehow, but I couldn't figure it out. I looked at modular multiplication algorithms, residue number systems, and then landed on Mersenne primes.

I then noticed that the modulus is `2^31 - 1` which is a Mersenne prime. But… not sure how that helps me.


In [66]:
import timeit
def generator_part1(start, factor):
    acc = start
    yield start
    while True:
        acc = (acc * factor) % 2147483647
        yield acc

In [67]:
def judge_part1_test():
    A = generator_part1(65, 16807)
    B = generator_part1(8921, 48271)
    n = sum([int(next(A) & 65535 == next(B) & 65535) for i in range(40_000_000)])
    print(n)
    
#timeit.timeit(judge_part1_test, number=1)

In [68]:
def judge_part1_input():
    A = generator_part1(116, 16807)
    B = generator_part1(299, 48271)
    n = sum([int(next(A) & 65535 == next(B) & 65535) for i in range(40_000_000)])
    print(n)
    
timeit.timeit(judge_part1_input, number=1)

569


57.98723808300201

## Part 2
> now they only hand a value to the judge when it meets their criteria:
> Generator A looks for values that are multiples of 4.
> Generator B looks for values that are multiples of 8.
> Each generator functions completely independently
> This change makes the generators much slower, and the judge is getting impatient; it is now only willing to consider 5 million pairs

In [69]:
def generator_part2(start, factor, multiple):
    acc = start
    yield start
    while True:
        acc = (acc * factor) % 2147483647
        if (acc % multiple == 0):
            yield acc

In [70]:
def judge_part2_test():
    A = generator_part2(65, 16807, 4)
    B = generator_part2(8921, 48271, 8)
    n = sum([int(next(A) & 65535 == next(B) & 65535) for i in range(5_000_000)])
    print(n)
    
#timeit.timeit(judge_part2_test, number=1)

In [71]:
def judge_part2_input():
    A = generator_part2(116, 16807, 4)
    B = generator_part2(299, 48271, 8)
    n = sum([int(next(A) & 65535 == next(B) & 65535) for i in range(5_000_000)])
    print(n)
    
timeit.timeit(judge_part2_input, number=1)

298


22.71304503900319

# Day 16
## Part 1
> There are sixteen programs in total, named a through p. They start by standing in a line: a stands in position 0, b stands in position 1, and so on until p, which stands in position 15.

> The programs' dance consists of a sequence of dance moves:

> - Spin, written sX, makes X programs move from the end to the front, but maintain their order otherwise. (For example, s3 on abcde produces cdeab).
> - Exchange, written xA/B, makes the programs at positions A and B swap places.
> - Partner, written pA/B, makes the programs named A and B swap places.

This seems straightforward, with the exception of Partner. Instead of a linear scan, perhaps we can maintain a map from program to index, updated after every move. This way, Partner can be implemented as Exchange. Python's list manipulation should make Spin a breeze. To avoid extra memory allocation, we want to use a list as a data structure, but not create a new list, especially on the spin operation.

In [72]:
import re
class PermutationPromenade:
    def __init__(self,init_list):
        "Initializes this class with the given programs in their initial order."
        self.len = len(init_list)
        # Maintain two lists, to avoid new list allocation when performing the spin op.
        # We maintain a current pointer to know which list is the current one.
        self.L1 = list(init_list)
        self.L2 = [None] * self.len
        # Maps from program name to position, to transform the partner op into the exchange op
        self.prog_map = {k:v for k, v in zip(init_list, range(self.len))}
        self.current = self.L1
        self.other = self.L2
        
    def spin(self, x):
        "Makes X programs move from the end to the front, but maintain their order otherwise."
        for i in range(self.len-x, self.len):
            self.other[i-self.len+x] = self.current[i]
        for i in range(0, self.len-x):
            self.other[i+x] = self.current[i]
        #print(self.other)
        self.current, self.other = self.other, self.current
        self.prog_map.update({k:v for k, v in zip(self.current, range(self.len))})
        #print(self.current)
    
    def exchange(self, posA, posB):
        "Makes the programs at positions A and B swap places"
        self.prog_map[self.current[posA]] = posB
        self.prog_map[self.current[posB]] = posA
        self.current[posA], self.current[posB] = self.current[posB], self.current[posA]
        #print(self.current)

    def partner(self, nameA, nameB):
        "Makes the programs named A and B swap places"
        posA = self.prog_map[nameA]
        posB = self.prog_map[nameB]
        return self.exchange(posA, posB)

    def parse_instrs(self, instrs):
        "Given a list of instructions, parse them and build an internal list that can be exec() or eval() later."
        self.parsed_instrs = []
        for instr in instrs:
            #print(instr)
            if instr[0] == 's':
                m = re.match(r"s(\d+)", instr)
                x = int(m[1])
                #self.spin(x)
                self.parsed_instrs.append('self.spin(%d)' % x)
            elif instr[0] == 'x':
                m = re.match(r"x(\d+)/(\d+)", instr)
                a = int(m[1])
                b = int(m[2])
                #self.exchange(a, b)
                self.parsed_instrs.append('self.exchange(%d,%d)' % (a,b))
            elif instr[0] == 'p':
                m = re.match(r"p(.)/(.)", instr)
                a = m[1]
                b = m[2]
                #self.partner(a, b)
                self.parsed_instrs.append('self.partner(\'%s\',\'%s\')' % (a,b))

    def execute_parsed(self):
        for instr in self.parsed_instrs:
            exec(instr)

In [73]:
test = PermutationPromenade(list('abcde'))
test.parse_instrs(['s1','x3/4', 'pe/b'])
test.execute_parsed()
''.join(test.current)

'baedc'

In [74]:
real = PermutationPromenade(list('abcdefghijklmnop'))
real.parse_instrs(Input(16).read().split(','))
real.execute_parsed()
''.join(real.current)

'ceijbfoamgkdnlph'

## Part 2
> Keeping the positions they ended up in from their previous dance, the programs perform it again and again: including the first dance, a total of one billion (1000000000) times.

> In what order are the programs standing after their billion dances?

After iterating for a while, I realized that executing the dance a billion times would take a long time. Unfortunately, I made the incorrect assumption that optimizing my code would help. I tried:

- Implementing the `spin` operation without additional memory allocation, by maintaining two lists, and copying from one to the other
- Parsing the set of dance moves once, storing them as a list of Python code strings, then replaying them on the list via `exec()`

I knew there had to be a trick to this puzzle, and it too me too long to realize that perhaps the set of dance moves resulted in a sequence of positions that repeated. By adding a cache, I was able to figure out the cycle length. Then, it's just a simple modular operation to determine which position we would be in after a billion iterations.

In [75]:
def find_cycle(init_list):
    "Given the initial list, perform the permutation dance repeatedly until a cycle is found. Return a list of positions indexed by the number of dances to arrive at that position; the size of the list is the size of the cycle."
    P = PermutationPromenade(init_list)
    instrs = Input(16).read().split(',')
    P.parse_instrs(instrs)
    cache = [''.join(init_list)]
    while True:
        P.execute_parsed()
        if ''.join(P.current) in cache:
            return cache
        cache.append(''.join(P.current))

In [76]:
cycles = find_cycle(list('abcdefghijklmnop'))
cycles[1_000_000_000 % 60]

'pnhajoekigcbflmd'

# Day 17
## Part 1
> It starts with a circular buffer containing only the value 0, which it marks as the current position. It then steps forward through the circular buffer some number of steps (your puzzle input) before inserting the first new value, 1, after the value it stopped on. The inserted value becomes the current position. Then, it steps forward from there the same number of steps, and wherever it stops, inserts after it the second new value, 2, and uses that as the new current position again.

> It repeats this process of stepping forward, inserting a new value, and using the location of the inserted value as the new current position a total of 2017 times, inserting 2017 as its final operation, and ending with a total of 2018 values (including 0) in the circular buffer.

In [77]:
def spinlock(steps, times):
    buf = [0]
    pos = 0
    for i in range(1,times+1):
        pos = (pos + steps % len(buf)) % len(buf)
        buf.insert(pos+1, i)
        pos += 1
    return buf

In [78]:
test = spinlock(3, 2017)
assert(test[(test.index(2017)+1) % len(test)]) == 638

In [79]:
real = spinlock(382, 2017)
real[(real.index(2017)+1) % len(real)]

1561

## Part 2

> The spinlock has just finished inserting its fifty millionth value (50000000).
> What is the value after 0 the moment 50000000 is inserted?

There are two key insights here: the first is that the 0 element is fixed. Since new elements are always inserted after a given position, 0 will never move. The second is that we don't need to keep extending a list to hold fifty million items, we just need to calculate the new current positions, and remember what value we (pretended to) insert there.

In [80]:
def angry_spinlock(steps, times):
    buf_len = 1 # Starts with [0]
    pos = 0
    insert_after_0 = None
    for i in range(1,times+1):
        pos = (pos + steps % buf_len) % buf_len
        if pos == 0:
            insert_after_0 = i
        buf_len += 1
        pos += 1
    return insert_after_0

In [81]:
angry_spinlock(382, 50_000_000)

33454823

# Day 18
## Part 1
> the assembly is meant to operate on a set of registers that are each named with a single letter and that can each hold a single integer. You suppose each register should start with a value of 0.
> - `snd X` plays a sound with a frequency equal to the value of X.
> - `set X Y` sets register X to the value of Y.
> - `add X Y` increases register X by the value of Y.
> - `mul X Y` sets register X to the result of multiplying the value contained in register X by the value of Y.
> - `mod X Y` sets register X to the remainder of dividing the value contained in register X by the value of Y (that is, it sets X to the result of X modulo Y).
> - `rcv X` recovers the frequency of the last sound played, but only when the value of X is not zero. (If it is zero, the command does nothing.)
> - `jgz X Y` jumps with an offset of the value of Y, but only if the value of X is greater than zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)

> Many of the instructions can take either a register (a single letter) or a number. The value of a register is the integer it contains; the value of a number is that number.
> After each jump instruction, the program continues with the instruction to which the jump jumped. After any other instruction, the program continues with the next instruction. Continuing (or jumping) off either end of the program terminates it.



In [82]:
import re
from collections import defaultdict
    
class Duet:
    def __init__(self, lines):
        self.lines = lines
        self.last_sound = None
        self.regs = defaultdict(lambda: 0)
        self.char_re = re.compile("[a-z]")
    
    def get_assembly_value(self, arg):
        "Given an assembly value, return the register value, if the argument is a register; otherwise, the value itself."
        if self.char_re.fullmatch(arg):
            return self.regs[arg]
        else:
            return int(arg)

    def run(self):
        next_line = 0
        while next_line in range(0, len(self.lines)):
            next_line = self.execute_line(next_line)
        print("Done execution")
            
    def execute_line(self, line_index):
        offset = 1
        line = self.lines[line_index]
        if line[0:3] == 'snd':
            arg = re.match(r"snd (.+)", line).groups()[0]
            val = self.get_assembly_value(arg)
            self.last_sound = val
        elif line[0:3] == 'set':
            reg, arg = re.match(r"set (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] = val
        elif line[0:3] == 'add':
            reg, arg = re.match(r"add (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] += val
        elif line[0:3] == 'mul':
            reg, arg = re.match(r"mul (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] *= val
        elif line[0:3] == 'mod':
            reg, arg = re.match(r"mod (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] %= val
        elif line[0:3] == 'rcv':
            arg = re.match(r"rcv (.+)", line).groups()[0]
            val = self.get_assembly_value(arg)
            if val != 0:
                print("Recovered value %d" % (self.last_sound))
                offset = 2*len(self.lines) # Really big offset to stop program
        elif line[0:3] == 'jgz':
            X, arg = re.match(r"jgz (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            check = self.get_assembly_value(X)
            if check > 0:
                offset = val
        
        return line_index + offset


In [83]:
test = """set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2"""

test_duet = Duet(test.split('\n'))

In [84]:
test_duet.run()

Recovered value 4
Done execution


In [85]:
real = Input(18).read().splitlines()
real_duet = Duet(real)
real_duet.run()

Recovered value 4601
Done execution


## Part 2
> This assembly code isn't about sound at all - it's meant to be run twice at the same time.

> Each running copy of the program has its own set of registers and follows the code independently - in fact, the programs don't even necessarily run at the same speed. To coordinate, they use the send (snd) and receive (rcv) instructions:

> `snd X` sends the value of X to the other program. These values wait in a queue until that program is ready to receive them. Each program has its own message queue, so a program can never receive a message it sent.

> `rcv X` receives the next value and stores it in register X. If no values are in the queue, the program waits for a value to be sent to it. Programs do not continue to the next instruction until they have received a value. Values are received in the order they are sent.

> Each program also has its own program ID (one 0 and the other 1); the register p should begin with this value.

> \[if\] no data is waiting for either of them, and they reach a deadlock. When this happens, both programs terminate.

> Once both of your programs have terminated (regardless of what caused them to do so), how many times did program 1 send a value?

Instead of actually implementing a blocking producer-consumer implementation, my plan is to execute steps of both programs one at a time using a "controller", and manually pass the values back and forth. The controller program will be responsible for receiving values and "restarting" a program that is waiting.

I debugged an infinite loop for a long time on this one, eventually turning to the [subreddit](https://www.reddit.com/r/adventofcode/) for help. I made a mistake that tripped many people up: not noticing that the `jgz` instruction allows register names _and_ values in _both_ arguments.

Two gotchas about Python that I learned:

- Python's syntax for a 1-element tuple is `(1,)`, i.e. don't forget the trailing comma
- To implement the queue of send/receive values, I used a list. `list.pop()` removes the **last** element, i.e. the item most recently added by `list.append()`. What I needed was `list.pop(0)`, which removes from the front.

In [86]:
class Duet2:
    def __init__(self, lines, program_id):
        self.lines = lines
        self.regs = defaultdict(lambda: 0)
        self.char_re = re.compile("[a-z]")
        self.regs['p'] = program_id
        self.id = program_id
    
    def get_assembly_value(self, arg):
        "Given an assembly value, return the register value, if the argument is a register; otherwise, the value itself."
        if self.char_re.fullmatch(arg):
            return self.regs[arg]
        else:
            return int(arg)

    def execute_line(self, line_index, received):
        "Parse the instruction specified by the line index, returning a tuple of \
        ('next', <next_line_num>), ('end'), ('wait') or ('send', <value>, <next_line_num>)" 
        status = 'next'
        offset = 1
        send_val = None
        line = self.lines[line_index]

        #print('{} executing {}, '.format(self.id, line), end='')
        
        if line[0:3] == 'snd':
            arg = re.match(r"snd (.+)", line).groups()[0]
            send_val = self.get_assembly_value(arg)
            status = 'send'
        elif line[0:3] == 'set':
            reg, arg = re.match(r"set (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] = val
        elif line[0:3] == 'add':
            reg, arg = re.match(r"add (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] += val
        elif line[0:3] == 'mul':
            reg, arg = re.match(r"mul (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] *= val
        elif line[0:3] == 'mod':
            reg, arg = re.match(r"mod (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] %= val
        elif line[0:3] == 'rcv':
            if len(received) != 0:
                reg = re.match(r"rcv (.+)", line).groups()[0]
                self.regs[reg] = received.pop(0)
            else:
                status = 'wait'
                offset = 0
        elif line[0:3] == 'jgz':
            X, arg = re.match(r"jgz (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            check = self.get_assembly_value(X)
            if check > 0:
                offset = val
        
        
        if (line_index + offset) >= len(self.lines):
            status = 'end'
            
        if status == 'next':
            return (status, line_index + offset)
        elif status == 'wait' or status == 'end':
            return (status,)
        elif status == 'send':
            return (status, send_val, line_index + offset)


In [87]:
real = Input(18).read().splitlines()

P0 = Duet2(real, 0)
P1 = Duet2(real, 1)

# Values sent from each program
P0sendQ = []
P1sendQ = []
# Result of each instruction execution
status0 = ('next', 0)
status1 = ('next', 0)
# Next instruction
P0next = status0[1] 
P1next = status1[1]
# Times P1 sent
P1send = 0
while True:
    if status0[0] != 'end':
        status0 = P0.execute_line(P0next, P1sendQ)
    if status1[0] != 'end':
        status1 = P1.execute_line(P1next, P0sendQ)
    
    #print("")
    #print("Status: 0: {}, 1: {}".format(status0, status1))
    if status0[0] == 'next':
        P0next = status0[1]
    elif status0[0] == 'end':
        pass
    elif status0[0] == 'wait':
        pass
    elif status0[0] == 'send':
        P0sendQ.append(status0[1])
        P0next = status0[2]
        
    if status1[0] == 'next':
        P1next = status1[1]
    elif status1[0] == 'end':
        pass
    elif status1[0] == 'wait':
        pass
    elif status1[0] == 'send':
        P1send += 1
        P1sendQ.append(status1[1])
        P1next = status1[2]
        
    if (status0[0] == 'wait' and status1[0] == 'wait') or \
       (status0[0] == 'wait' and status1[0] == 'end') or \
       (status0[0] == 'end' and status1[0] == 'wait') or \
       (status0[0] == 'end' and status1[0] == 'end'):
        break
    

In [88]:
P1send

6858

# Day 19
## Part 1
> Lines (drawn with |, -, and +) show the path it needs to take, starting by going down onto the only line connected to the top of the diagram. It needs to follow this path until it reaches the end (located somewhere within the diagram) and stop there.

> Sometimes, the lines cross over each other; in these cases, it needs to continue going the same direction, and only turn left or right when there's no other option. In addition, someone has left letters on the line
> What letters will it see (in the order it would see them) if it follows the path?

My approach is going to be reading in the entire diagram into a list of lists. Keep track of the current direction, and wander through the array. If the same symbol is seen, continue in the same direction. If a letter is seen, add it to a list. If a `+` is seen, look ahead in the same direction to see if we can keep going, or if we're at the edge; if not, look left then right to see if path continues there and change direction. If the opposite symbol is seen, we're at a crossing, and should continue in same direction.

## Part 2
> The packet is curious how many steps it needs to go.

In [89]:
import string
def find_start(grid):
    "Given a list of lists representing the diagram, find the coordinate of the first | when scanning left to right, top to bottom."
    height = len(grid)
    width = len(grid[0])
    for j in range(height):
        for i in range(width):
            if grid[j][i] == '|':
                return (i,j)

def within_grid(coord, grid_width, grid_height):
    "Returns whether the given coord is within the grid boundaries."
    (i,j) = coord
    return i in range(0, grid_width) and j in range(0, grid_height)

def traverse_map(grid):
    "Given a list of lists representing the diagram, find the start, and follow it, returning the letters seen in order as a single string."
    gw, gh = len(grid[0]), len(grid)
    (i, j) = find_start(grid)
    dir = 's'
    letters = []
    steps = 0
    while True:
        #print("{} {}".format(grid[j][i], (i,j)), end='')
        next_dir = dir
        sym = grid[j][i]
        if sym == '|':
            steps += 1
            if dir == 's' or dir == 'n':
                pass
            elif dir == 'e' or dir == 'w':
                #print("Crossing at {}, continuing".format((i,j)), end='')
                pass
        elif sym == '-':
            steps += 1
            if dir == 's' or dir == 'n':
                #print("Crossing at {}, continuing".format((i,j)), end='')
                pass
            elif dir == 'e' or dir == 'w':
                pass
        elif sym == '+':
            steps += 1
            if dir in ['n', 's']:
                if within_grid((i+1,j), gw, gh) and grid[j][i+1] == '-':
                    next_dir = 'e'
                elif within_grid((i-1, j), gw, gh) and grid[j][i-1] == '-':
                    next_dir = 'w'
            elif dir in ['e', 'w']:
                if within_grid((i,j+1), gw, gh) and grid[j+1][i] == '|':
                    next_dir = 's'
                elif within_grid((i, j-1), gw, gh) and grid[j-1][i] == '|':
                    next_dir = 'n'
                   
        elif sym in string.ascii_uppercase:
            steps += 1
            letters.append(sym)
            #print("Saw %s" % sym)
            # dir stays the same
        else:
            return ''.join(letters), steps

        if dir != next_dir:
            #print("Changing dir from {} to {}".format(dir, next_dir), end='')
            dir = next_dir
            
        if dir == 's':
            j += 1
        elif dir == 'n':
            j -= 1
        elif dir == 'e':
            i += 1
        elif dir == 'w':
            i -= 1


In [90]:
grid = [list(line) for line in Input(19).read().splitlines()]
traverse_map(grid)

('GINOWKYXH', 16636)

# Day 20
## Part 1
> a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the X, Y, and Z coordinates for the particle's position (p), velocity (v), and acceleration (a), each in the format <X,Y,Z>.

> Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:

> - Increase the X velocity by the X acceleration.
> - Increase the Y velocity by the Y acceleration.
> - Increase the Z velocity by the Z acceleration.
> - Increase the X position by the X velocity.
> - Increase the Y position by the Y velocity.
> - Increase the Z position by the Z velocity.

> which particle will stay closest to position <0,0,0> in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's X, Y, and Z position.

Running the simulation, then outputting a sorted list of the five nearest particles, then seeing if their rankings start to stabilize, is my first guess at how to model this.

I learned to be carefully about naming variables after reserved words, and built-in functions, e.g. `list` or `map`. If you do, you'll get hard-to-debug errors!

In [91]:
from operator import add
class Particle:
    def __init__(self, particle_id, pos, vel, acc):
        self.id = particle_id
        self.pos = pos
        self.vel = vel
        self.acc = acc
        
    def update(self):
        self.vel = list(map(add, self.vel, self.acc))
        self.pos = list(map(add, self.pos, self.vel))
        
    def distance_from(self, other):
        return sum([abs(other[i] - self.pos[i]) for i in [0,1,2]])

In [92]:
def swarm(lines, iters):
    particles = {}
    particle_id = 0
    for line in lines:
        g = re.match(r"p=<(-?\d+),(-?\d+),(-?\d+)>, v=<(-?\d+),(-?\d+),(-?\d+)>, a=<(-?\d+),(-?\d+),(-?\d+)>", line).groups()
        pos = [int(g[0]), int(g[1]), int(g[2])]
        vel = [int(g[3]), int(g[4]), int(g[5])]
        acc = [int(g[6]), int(g[7]), int(g[8])]
        particles[particle_id] = Particle(particle_id, pos, vel, acc)
        particle_id += 1
    for i in range(iters):
        for P in particles.values():
            P.update()

    return particles

In [93]:
real = Input(20).readlines()

In [94]:
P = swarm(real, 700)
out = sorted(P.values(), key=lambda P: P.distance_from([0,0,0]))
[x.id for x in out[0:5]]

[364, 485, 205, 46, 396]

In [95]:
P = swarm(real, 1000)
out = sorted(P.values(), key=lambda P: P.distance_from([0,0,0]))
[x.id for x in out[0:5]]

[364, 485, 205, 46, 396]

## Part 2
> Particles collide if their positions ever exactly match. Because particles are updated simultaneously, more than two particles can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.
> How many particles are left after all collisions are resolved?

Keep a map of position to particles. Update as usual, adding all particles to map. On each pass through, if we know that a position has more than one particle, remove them from the map. Run a few times with increasing iterations to see when we reach a steady state.

In [96]:
from collections import defaultdict

def handle_collisions(lines, iters):
    space = defaultdict(lambda: []) # Map from location tuple to particle IDs
    particles = swarm(lines, 0)

    for i in range(iters):
        collision_locations = []
        for P in particles.values():
            P.update()
            new_pos = tuple(P.pos)
            space[new_pos].append(P.id)
            if len(space[new_pos]) > 1:
                collision_locations.append(new_pos)
        if len(collision_locations) > 0:
            for loc in collision_locations:
                tbd = space[loc]
                for particle_id in tbd:
                    try:
                        del particles[particle_id]
                    except KeyError:
                        pass
                    
    return len(particles)

In [97]:
handle_collisions(real, 700)

420

In [98]:
handle_collisions(real, 1000)

420

# Day 21
## Part 1
> The image consists of a two-dimensional square grid of pixels that are either on (#) or off (.). The program always begins with this pattern:
```
.#.
..#
###
```
> Because the pattern is both 3 pixels wide and 3 pixels tall, it is said to have a **size** of 3.

> Then, the program repeats the following process:

> - If the size is evenly divisible by 2, break the pixels up into 2x2 squares, and convert each 2x2 square into a 3x3 square by following the corresponding enhancement rule.
> - Otherwise, the size is evenly divisible by 3; break the pixels up into 3x3 squares, and convert each 3x3 square into a 4x4 square by following the corresponding enhancement rule.

> Because each square of pixels is replaced by a larger one, the image gains pixels and so its **size** increases.

> The artist's book of enhancement rules is nearby (your puzzle input); however, it seems to be missing rules. The artist explains that sometimes, one must **rotate** or **flip** the input pattern to find a match. (Never rotate or flip the output pattern, though.) Each pattern is written concisely: rows are listed as single units, ordered top-down, and separated by slashes.

Some thoughts:

- `numpy` arrays have a lot of methods for manipulating matrices and we'll need to flip and rotate, and combine and split
- We'll need methods to turn the concise enhancement rules into matrices and v.v.

In [100]:
import numpy as np

def matrix_from_pattern(patt):
    rows = patt.split('/')
    return np.matrix([list(row) for row in rows])

def pattern_from_matrix(A):
    return '/'.join([''.join(row) for row in A.tolist()])

In [101]:
def make_rotates_and_flips(A):
    "Return a set of A itself, its flips (horiz, vert, horiz+vert) and three rotations (90, 180, 270) and their combination"
    return set([pattern_from_matrix(M) for M in [A, np.fliplr(A), np.flipud(A), np.rot90(A), np.flipud(np.rot90(A)), np.fliplr(np.rot90(A)), np.rot90(A, 2), np.rot90(A, 3)]])

In [102]:
import re

def parse_fractal_rules(lines):
    "Add each rule to a dictionary, and pre-compute the rotates/flips for each input."
    rules = {}
    for line in lines:
        inp, out = re.match(r"([./#]+) => ([./#]+)", line).groups()
        xforms = make_rotates_and_flips(matrix_from_pattern(inp))
        for x in xforms:
            rules[x] = out
    return rules

def enhancement_matrix(rules, patt):
    "Given a pattern representing a matrix, returns the output matrix based on the enhancement rules."
    #xformpatts = make_rotates_and_flips(matrix_from_pattern(patt))
    #for x in xformpatts:
    if patt in rules:
        return matrix_from_pattern(rules[patt])
    print("No rule for " + patt)

In [103]:
def fractal_art(rulelines, startpatt, iters=5):
    rules = parse_fractal_rules(rulelines)
    A = matrix_from_pattern(startpatt)
    for i in range(iters):
        #print(i, flush=True)
        rows, cols = A.shape
        if rows % 2 == 0:
            newrows = newcols = rows//2*3
            sz = 2
        else:
            newrows = newcols = rows//3*4
            sz = 3
        newA = np.full((newrows,newcols), '.')
        for r in range(0, rows//sz):
            for c in range(0, cols//sz):
                inpatt = pattern_from_matrix(A[r*sz:(r+1)*sz, c*sz:(c+1)*sz])
                outmatrix = enhancement_matrix(rules, inpatt)
                outr, outc = outmatrix.shape
                newA[r*outr:(r+1)*outr, c*outc:(c+1)*outc] = outmatrix
        A = newA
    return np.count_nonzero(A == '#')

In [104]:
sample = """../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#"""

assert(fractal_art(sample.split('\n'), '.#./..#/###', 2)) == 12

In [105]:
real = Input(21).read().splitlines()
fractal_art(real, '.#./..#/###')

147

After trying to run Part 2 on my original solution, it was taking too long, so I thought of optimizations. I realized that I didn't have to apply the flip/rotate transformations on the submatrices, I could pre-compute them on the rules themselves. With that implemented, it took about 20 seconds to calculate the answer.

For further optimization, I could potentially try not using a string datatype in the `numpy` arrays, and 

In [106]:
fractal_art(real, '.#./..#/###', 18)

1936582

# Day 22
## Part 1
> The grid computing cluster is a seemingly-infinite two-dimensional grid of compute nodes. Each node is either **clean** or **infected** by the virus.

> exactly one **virus carrier** moves through the network, infecting or cleaning nodes as it moves. The virus carrier is always located on a single node in the network (the **current node**) and keeps track of the **direction** it is facing.

> the virus carrier works in bursts; in each burst, it **wakes up**, does some **work**, and goes back to **sleep**. The following steps are all executed **in order** one time each burst:

> - If the **current node** is **infected**, it turns to its **right**. Otherwise, it turns to its **left**. (Turning is done in-place; the **current node** does not change.)
> - If the **current node** is **clean**, it becomes **infected**. Otherwise, it becomes **cleaned**. (This is done **after** the node is considered for the purposes of changing direction.)
> - The virus carrier moves **forward** one node in the direction it is facing.

> Diagnostics have also provided a map of the node infection status (your puzzle input). Clean nodes are shown as `.`; infected nodes are shown as `#`. The virus carrier begins in the middle of the map facing **up**.

> Given your actual map, after 10000 bursts of activity, how many bursts cause a node to become infected? (Do not count nodes that begin infected.)

Modeling an infinite grid could be done with a sparse array implementation, of which scipy has a few. But I'm going to try using a hash whose keys are tuples representing the coordinates as integers. I'll arbitrarily define the centre grid element of the input map as `(0,0)`. 

In [157]:
def sporifica(G, bursts=10000):
    # 2-dimensional grid, indexed by (i,j) tuples
    grid = defaultdict(lambda: '.')
    # DIRS: UP, RIGHT, DOWN, LEFT
    DIRS = [(0, -1), (1, 0), (0, 1), (-1, 0)]

    # Input is an array of strings, representing the input grid
    def populate_grid(G):
        n = len(G)
        assert(n == len(G[0])) # Check square input

        for j in range(n):
            for i in range(n):
                grid[(i - n//2, j - n//2)] = G[j][i]
    
    def run(bursts):
        dir = 0
        burst_infected = 0
        cur = (0,0)
        dirsize = len(DIRS)
        for _ in range(bursts):
            if grid[cur] == '#':
                dir = (dir + 1) % dirsize
                grid[cur] = '.'
            else:
                dir = (dir - 1) % dirsize
                grid[cur] = '#'
                burst_infected += 1
            cur = (cur[0] + DIRS[dir][0], cur[1] + DIRS[dir][1])
        return burst_infected
    
    populate_grid(G)
    return run(bursts)

In [158]:
test = "..#\n#..\n...".split('\n')
assert(sporifica(test, 10000) == 5587)

In [159]:
sporifica(Input(22).read().splitlines(), 10000)

5433

## Part 2
> Now, before it infects a clean node, it will **weaken** it to disable your defenses. If it encounters an infected node, it will instead **flag** the node to be cleaned in the future. So:

> - **Clean** nodes become **weakened**.
> - **Weakened** nodes become **infected**.
> - **Infected** nodes become **flagged**.
> - **Flagged** nodes become **clean**.

> Every node is always in exactly one of the above states.

> the following logic during its bursts of action:

> - Decide which way to turn based on the current node:
>     - If it is clean, it turns left.
>     - If it is weakened, it does not turn, and will continue moving in the same direction.
>     - If it is infected, it turns right.
>     - If it is flagged, it reverses direction, and will go back the way it came.
> - Modify the state of the current node, as described above.
> - The virus carrier moves forward one node in the direction it is facing.

> the same initial state as the previous example, and drawing weakened as W and flagged as F

In [164]:
def sporifica2(G, bursts=10000):
    # 2-dimensional grid, indexed by (i,j) tuples
    grid = defaultdict(lambda: '.')
    # DIRS: UP, RIGHT, DOWN, LEFT
    DIRS = [(0, -1), (1, 0), (0, 1), (-1, 0)]

    # Input is an array of strings, representing the input grid
    def populate_grid(G):
        n = len(G)
        assert(n == len(G[0])) # Check square input

        for j in range(n):
            for i in range(n):
                grid[(i - n//2, j - n//2)] = G[j][i]
    
    def run(bursts):
        dir = 0
        burst_infected = 0
        cur = (0,0)
        dirsize = len(DIRS)
        for _ in range(bursts):
            x = grid[cur]
            if x == '.': # clean
                dir = (dir - 1) % dirsize
                grid[cur] = 'W'
            elif x == 'W': # weakened
                grid[cur] = '#'
                burst_infected += 1
            elif x == '#': # infected
                dir = (dir + 1) % dirsize
                grid[cur] = 'F'
            elif x == 'F': # flagged
                dir = (dir + 2) % dirsize
                grid[cur] = '.'

            cur = (cur[0] + DIRS[dir][0], cur[1] + DIRS[dir][1])

        return burst_infected
    
    populate_grid(G)
    return run(bursts)

In [165]:
test = "..#\n#..\n...".split('\n')
assert(sporifica2(test, 100) == 26)

In [166]:
sporifica2(Input(22).read().splitlines(), 10_000_000)

2512599

# Day 23
## Part 1
> The code it's running seems to be a variant of the kind you saw recently on that tablet. The general functionality seems very similar, but some of the instructions are different:

> - `set X Y` sets register X to the value of Y.
> - `sub X Y` decreases register X by the value of Y.
> - `mul X Y` sets register X to the result of multiplying the value contained in register X by the value of Y.
> - `jnz X Y` jumps with an offset of the value of Y, but only if the value of X is not zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)

> Only the instructions listed above are used. The eight registers here, named a through h, all start at 0.

> how many times is the mul instruction invoked?

In [168]:
class Conflagration:
    def __init__(self, lines):
        self.lines = lines
        self.regs = defaultdict(lambda: 0)
        self.char_re = re.compile("[a-z]")
        self.mul_exe = 0
    
    def get_assembly_value(self, arg):
        "Given an assembly value, return the register value, if the argument is a register; otherwise, the value itself."
        if self.char_re.fullmatch(arg):
            return self.regs[arg]
        else:
            return int(arg)

    def run(self):
        next_line = 0
        while next_line in range(0, len(self.lines)):
            next_line = self.execute_line(next_line)
        print("mul call count: %d" % self.mul_exe)
            
    def execute_line(self, line_index):
        offset = 1
        line = self.lines[line_index]
        if line[0:3] == 'set':
            reg, arg = re.match(r"set (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] = val
        elif line[0:3] == 'sub':
            reg, arg = re.match(r"sub (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] -= val
        elif line[0:3] == 'mul':
            reg, arg = re.match(r"mul (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            self.regs[reg] *= val
            self.mul_exe += 1
        elif line[0:3] == 'jnz':
            X, arg = re.match(r"jnz (.) (.+)", line).groups()
            val = self.get_assembly_value(arg)
            check = self.get_assembly_value(X)
            if check != 0:
                offset = val
        
        return line_index + offset


In [170]:
prog = Input(23).read().splitlines()
Conflagration(prog).run()


mul call count: 3969
